# STEP
1. Modello (DistillBERT + un altro modello (?) ) per i dati in inglese, fare training + finetuning.
2. Stesso modello, ma sui dati in italiano + finetuning.
3. Traduzione ITA-ENG:
  * trovare modello già fatto su github
  * from scratch
    * ref: https://statics.teams.cdn.office.net/evergreen-assets/safelinks/1/atp-safelinks.html , <br> https://statics.teams.cdn.office.net/evergreen-assets/safelinks/1/atp-safelinks.html
4. Test modello (1) sui dati italiani tradotti (3).

In [1]:
# Various pip
!pip install tensorflow
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.9 MB/s eta 0:00:00


In [2]:
# Dataframe management
import pandas as pd

# Data manipulation
import numpy as np
import random

# NNs
from tensorflow import keras
import tensorflow as tf

# Transformers
import transformers

# Convert to HuggingFace dataset
import pyarrow as pa
from datasets import Dataset
import datasets

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset Analysis

In [30]:
# Get dataset
df_en = pd.read_csv('/content/drive/My Drive/NLP_HATESPEECH/data/mainDataEN.csv')
df_ita_m = pd.read_csv('/content/drive/My Drive/NLP_HATESPEECH/misogyny_data/AMI2020_TrainingSet/AMI2020_training_raw.tsv', sep="\t")
df_ita_hs = pd.read_csv('/content/drive/My Drive/NLP_HATESPEECH/haspeede2_dev/haspeede2_dev_taskAB.tsv', sep="\t")

In [27]:
df_en.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [85]:
df_en = df_en.drop(['Unnamed: 0'], axis=1)
#df_en

In [81]:
def is_hate_speech(value):
  if value['class'] == 2:
    return 0 #no hate speech
  else:
     return 1 #hate speech

df_en['hate_speech'] = df_en.apply(is_hate_speech, axis=1)

df_en.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,1,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,1,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,1,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,1,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [31]:
df_ita_m.head()

,id,text,misogynous,aggressiveness
0,1,@KassemAmin4 @Laylasexgdr Fatti trovare te lo...,1,1
1,2,@meb Tu dovresti ricominciare dai semafori a f...,1,1
2,3,"Amore,sei presentabile? Xchè così via Skype ti...",1,1
3,4,"@Il_nulla Salvo poi mandare la culona a Mosca,...",1,0
4,5,@GiorgiaMeloni @FratellidItaIia Vediamo Gentil...,1,1


In [33]:
#mysogynous tweets
df_ita_m[df_ita_m['misogynous']==1].count()

id                2337
text              2337
misogynous        2337
aggressiveness    2337
dtype: int64

In [34]:
#non mysogynous tweets
df_ita_m[df_ita_m['misogynous']==0].count()

id                2663
text              2663
misogynous        2663
aggressiveness    2663
dtype: int64

In [78]:
#merge 'misogynous' and 'aggressiveness' column into a new hate speech column.
df_ita_m['hate_speech'] = df_ita_m['misogynous'] | df_ita_m['aggressiveness']
df_ita_m = df_ita_m.drop(['misogynous', 'aggressiveness'], axis=1)

In [79]:
df_ita_m.head()

,id,text,hate_speech
0,1,@KassemAmin4 @Laylasexgdr Fatti trovare te lo...,1
1,2,@meb Tu dovresti ricominciare dai semafori a f...,1
2,3,"Amore,sei presentabile? Xchè così via Skype ti...",1
3,4,"@Il_nulla Salvo poi mandare la culona a Mosca,...",1
4,5,@GiorgiaMeloni @FratellidItaIia Vediamo Gentil...,1


In [77]:
#stereotypes are considered as hate speech
df_ita_hs['hate_speech'] = df_ita_hs['stereotype'] | df_ita_hs['hs']
df_ita_hs = df_ita_hs.drop(['stereotype', 'hs'], axis=1)

In [80]:
df_ita_hs.head()

,id,text,hate_speech
0,2066,"È terrorismo anche questo, per mettere in uno ...",0
1,2045,@user @user infatti finché ci hanno guadagnato...,0
2,61,"Corriere: Tangenti, Mafia Capitale dimenticata...",0
3,1259,"@user ad uno ad uno, perché quando i migranti ...",0
4,949,Il divertimento del giorno? Trovare i patrioti...,0
